In [71]:
import requests
from bs4 import BeautifulSoup

def fetch_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = ' '.join([p.text for p in soup.find_all('p')]) #extracting the text from the paragraphs
    return text

# Example usage
url = "https://www.digi24.ro/stiri/externe/congresul-a-aprobat-ajutorul-militar-critic-de-61-de-miliarde-pentru-ucraina-2767383"
article_text = fetch_article(url)
print(article_text)

Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, sâmbătă seară, de Camera Reprezentanților din Congresul SUA, după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front, relatează BBC. Este un moment de cotitură pentru Ucraina, a cărei situație pe front era tot mai dificilă și mai vulnerabilă. Rezultatul final al votului a fost de 311 la 112, voturile contra fiind ale republicanilor trumpiști. Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului. Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare, urmând să fie, apoi, promulgat de președintele Joe Biden. Conform oficialilor americani familiarizați cu subiectul, o parte din ajutoare vor ajunge pe frontul din Ucraina în cel mult o săptămână. Preşedintele Joe Biden a salutat, sâmbătă, adoptarea de către Camera Repre

In [72]:
import re
import nltk
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
import spacy

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('romanian'))
nlp = spacy.load('ro_core_news_sm')

def preprocess_text(text):
    # Adjust regex patterns to remove unwanted text
    patterns = [
        r'\bData publicării:.*?\d{2}:\d{2}\b',
        r'\bFoto:[^\n]+\b',
        r'Editor\s*:[^\n]+\b',
        r'Etichete:[^\n]+',
        r'Urmărește știrile[^\n]+',
        r'\b(Top Citite|Top Articole)[^\n]+',
        r'Sursa:[^\n]+',
        r'([Aa]scultă|Citește) și:[^\n]+',
        r'[\r\n]+\s*[\r\n]+',
        r'\bVezi și:[^\n]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.DOTALL)
    
    # Use custom tokenizer to split on full stops, commas, and other punctuation marks
    tokenizer = PunktSentenceTokenizer(train_text=text)
    sentences = tokenizer.tokenize(text)
    
    # Further split on commas to reduce sentence length and increase relevance
    smaller_sentences = []
    for sentence in sentences:
        smaller_sentences.extend(sentence.split(','))
    
    # Remove the sentences that have less than 5 words THAT ARE NOT STOP WORDS AND does not contain a verb or auxiliar verb, using spaCy
    smaller_sentences = [sentence for sentence in smaller_sentences if len([aux_word for aux_word in (sentence.split()) if aux_word not in stop_words]) >= 5 or any(token.pos_ in ['VERB', 'AUX'] for token in nlp(sentence))]
        
    return [sent.strip() for sent in smaller_sentences if sent.strip()]

# Example usage with a provided article text
processed_text = preprocess_text(article_text)
print("Text preprocesat: ", processed_text)
print("Text original: ", article_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stoic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stoic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Text preprocesat:  ['Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia', 'a fost aprobat', 'după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front', 'relatează BBC.', 'Este un moment de cotitură pentru Ucraina', 'a cărei situație pe front era tot mai dificilă și mai vulnerabilă.', 'Rezultatul final al votului a fost de 311 la 112', 'voturile contra fiind ale republicanilor trumpiști.', 'Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului.', 'Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare', 'urmând să fie', 'promulgat de președintele Joe Biden.', 'Conform oficialilor americani familiarizați cu subiectul', 'o parte din ajutoare vor ajunge pe frontul din Ucraina în cel mult o săptămână.', 'Preşedintele Joe Biden a salutat', 'Pachetul de 95 de miliarde de dolari „trimite un mesaj clar despre puterea leadership-ului 

FIRST APPROACH WITH RANKING SENTENCES WITH A CUSTOM SCORE SYSTEM

In [73]:
from collections import defaultdict
import heapq

def frequency_based_summarization(text, num_sentences=3):
    sentences = preprocess_text(text)
    words = [word_tokenize(sent.lower()) for sent in sentences]
    word_frequencies = defaultdict(int)
    for sentence in words:
        for word in sentence:
            if word not in stop_words and word.isalpha():
                word_frequencies[word] += 1

    # Calculating the scores for each sentence
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_frequencies[word]
                    # Increase the weight of sentences that appear at the beginning or the end of the article
                    if i < len(sentences) * 0.1 or i > len(sentences) * 0.9:
                        sentence_scores[sentence] *= 1.5
                else:
                    sentence_scores[sentence] += word_frequencies[word]

    # Selecting the best 'num_sentences' sentences
    summary_sentences = heapq.nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

# Example usage
summary = frequency_based_summarization(article_text)
print("Sumar: ", summary)
print("Text original: ", article_text)

Sumar:  Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare Camera Reprezentanţilor a adoptat și un nou pachet de sprijin militar de mai multe miliarde de dolari pentru Israel Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia
Text original:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, sâmbătă seară, de Camera Reprezentanților din Congresul SUA, după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front, relatează BBC. Este un moment de cotitură pentru Ucraina, a cărei situație pe front era tot mai dificilă și mai vulnerabilă. Rezultatul final al votului a fost de 311 la 112, voturile contra fiind ale republicanilor trumpiști. Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului. Este de așteptat ca Senatul SUA să aprobe și el pache

SECOND APPROACH WITH TF-IDF SCORE SYSTEM

In [74]:
# Count words in a sentence
def cnt_words(sentence):
    cnt = 0
    words = word_tokenize(sentence)
    for word in words:
        if word not in stop_words:
            cnt += 1
    return cnt

# Count the number of words for each sentence
def cnt_in_sentences(sentences):
    data = []
    i = 0
    for sentence in sentences:
        i += 1
        cnt = cnt_words(sentence)
        aux = {"id": i, "sentence": sentence, "cnt": cnt}
        data.append(aux)
    return data

cnt_in_sentences(processed_text)

[{'id': 1,
  'sentence': 'Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia',
  'cnt': 7},
 {'id': 2, 'sentence': 'a fost aprobat', 'cnt': 1},
 {'id': 3,
  'sentence': 'după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front',
  'cnt': 11},
 {'id': 4, 'sentence': 'relatează BBC.', 'cnt': 3},
 {'id': 5, 'sentence': 'Este un moment de cotitură pentru Ucraina', 'cnt': 4},
 {'id': 6,
  'sentence': 'a cărei situație pe front era tot mai dificilă și mai vulnerabilă.',
  'cnt': 6},
 {'id': 7,
  'sentence': 'Rezultatul final al votului a fost de 311 la 112',
  'cnt': 5},
 {'id': 8,
  'sentence': 'voturile contra fiind ale republicanilor trumpiști.',
  'cnt': 6},
 {'id': 9,
  'sentence': 'Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului.',
  'cnt': 9},
 {'id': 10,
  'sentence': 'Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele urm

In [75]:
def frequency_dictionaries(sentences):
    i = 0
    freq_list = []
    for sentence in sentences:
        i += 1
        words = word_tokenize(sentence)
        freq_dict = {}
        for word in words:
            if word not in stop_words and word.isalpha():
                if word in freq_dict:
                    freq_dict[word] += 1
                else:
                    freq_dict[word] = 1
        aux = {"id": i, "sentence": sentence, "freq": freq_dict}
        freq_list.append(aux)
    return freq_list

# Example usage
list_of_freq_dicts = frequency_dictionaries(processed_text)
print(list_of_freq_dicts)

[{'id': 1, 'sentence': 'Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia', 'freq': {'Pachetul': 1, 'ajutor': 1, 'militar': 1, 'Ucraina': 1, 'războiul': 1, 'apărare': 1, 'Rusia': 1}}, {'id': 2, 'sentence': 'a fost aprobat', 'freq': {'aprobat': 1}}, {'id': 3, 'sentence': 'după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front', 'freq': {'luni': 1, 'întregi': 1, 'tergiversări': 1, 'și': 1, 'contextul': 1, 'situații': 1, 'extrem': 1, 'complicate': 1, 'apărătorii': 1, 'ucraineni': 1, 'front': 1}}, {'id': 4, 'sentence': 'relatează BBC.', 'freq': {'relatează': 1, 'BBC': 1}}, {'id': 5, 'sentence': 'Este un moment de cotitură pentru Ucraina', 'freq': {'Este': 1, 'moment': 1, 'cotitură': 1, 'Ucraina': 1}}, {'id': 6, 'sentence': 'a cărei situație pe front era tot mai dificilă și mai vulnerabilă.', 'freq': {'situație': 1, 'front': 1, 'dificilă': 1, 'și': 1, 'vulnerabilă': 1}}, {'id': 7, 'sentence': 'Rez

In [76]:
def calculate_TF(data, freq_list):
    tf_scores = []
    for sentence in freq_list:
        id = sentence["id"]
        for word in sentence["freq"]:
            aux = {"id": id, "word": word, "TF": sentence["freq"][word] / data[id-1]["cnt"]}
            tf_scores.append(aux)
    return tf_scores

# Example usage
tf_scores = calculate_TF(cnt_in_sentences(processed_text), list_of_freq_dicts)
print(tf_scores)

[{'id': 1, 'word': 'Pachetul', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'ajutor', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'militar', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'Ucraina', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'războiul', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'apărare', 'TF': 0.14285714285714285}, {'id': 1, 'word': 'Rusia', 'TF': 0.14285714285714285}, {'id': 2, 'word': 'aprobat', 'TF': 1.0}, {'id': 3, 'word': 'luni', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'întregi', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'tergiversări', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'și', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'contextul', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'situații', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'extrem', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'complicate', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'apărătorii', 'TF': 0.09090909090909091}, {'id': 3, 'word': 'ucraineni', 'TF': 0.090

In [77]:
import math

def calculate_IDF(data, freq_list):
    idf_scores = []
    cnt = 0 
    for sentence in freq_list:
        cnt += 1
        for word in sentence["freq"]:
            val = sum([1 for sentence in freq_list if word in sentence["freq"]]) # Count the number of sentences that contain the word
            aux = {"id": cnt, "word": word, "IDF": math.log(len(data) / (1 + val))}
            idf_scores.append(aux)
    return idf_scores

# Example usage
idf_scores = calculate_IDF(cnt_in_sentences(processed_text), list_of_freq_dicts)
print(idf_scores)

[{'id': 1, 'word': 'Pachetul', 'IDF': 2.0794415416798357}, {'id': 1, 'word': 'ajutor', 'IDF': 1.791759469228055}, {'id': 1, 'word': 'militar', 'IDF': 2.0794415416798357}, {'id': 1, 'word': 'Ucraina', 'IDF': 1.791759469228055}, {'id': 1, 'word': 'războiul', 'IDF': 2.4849066497880004}, {'id': 1, 'word': 'apărare', 'IDF': 2.4849066497880004}, {'id': 1, 'word': 'Rusia', 'IDF': 2.4849066497880004}, {'id': 2, 'word': 'aprobat', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'luni', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'întregi', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'tergiversări', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'și', 'IDF': 1.5686159179138452}, {'id': 3, 'word': 'contextul', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'situații', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'extrem', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'complicate', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'apărătorii', 'IDF': 2.4849066497880004}, {'id': 3, 'word': 'ucraineni

In [78]:
def calculate_TFIDF(tf_scores, idf_scores):
    tfidf_scores = []
    for tf in tf_scores:
        for idf in idf_scores:
            if tf["id"] == idf["id"] and tf["word"] == idf["word"]:
                aux = {"id": tf["id"], "word": tf["word"], "TFIDF": tf["TF"] * idf["IDF"]}
                tfidf_scores.append(aux)
    return tfidf_scores

# Example usage
tfidf_scores = calculate_TFIDF(tf_scores, idf_scores)
print(tfidf_scores)

[{'id': 1, 'word': 'Pachetul', 'TFIDF': 0.29706307738283366}, {'id': 1, 'word': 'ajutor', 'TFIDF': 0.25596563846115067}, {'id': 1, 'word': 'militar', 'TFIDF': 0.29706307738283366}, {'id': 1, 'word': 'Ucraina', 'TFIDF': 0.25596563846115067}, {'id': 1, 'word': 'războiul', 'TFIDF': 0.35498666425542863}, {'id': 1, 'word': 'apărare', 'TFIDF': 0.35498666425542863}, {'id': 1, 'word': 'Rusia', 'TFIDF': 0.35498666425542863}, {'id': 2, 'word': 'aprobat', 'TFIDF': 2.4849066497880004}, {'id': 3, 'word': 'luni', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'întregi', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'tergiversări', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'și', 'TFIDF': 0.14260144708307684}, {'id': 3, 'word': 'contextul', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'situații', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'extrem', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'complicate', 'TFIDF': 0.22590060452618185}, {'id': 3, 'word': 'apărătorii', 'TFIDF'

In [79]:
def sentence_score(tfidf_scores, sentences, data):
    sentences_data = []
    for sentence_word_counter in data:
        score = 0
        for i in range(len(tfidf_scores)):
            word_data_from_tfidf = tfidf_scores[i]
            if word_data_from_tfidf["id"] == sentence_word_counter["id"]:
                score += word_data_from_tfidf["TFIDF"]
        aux = {"id": sentence_word_counter["id"], "sentence": sentences[sentence_word_counter["id"]-1], "score": score}
        sentences_data.append(aux)
    return sentences_data

# Example usage
sentence_scores = sentence_score(tfidf_scores, processed_text, cnt_in_sentences(processed_text))
print(sentence_scores)

[{'id': 1, 'sentence': 'Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia', 'score': 2.171017424454255}, {'id': 2, 'sentence': 'a fost aprobat', 'score': 2.4849066497880004}, {'id': 3, 'sentence': 'după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front', 'score': 2.3647470279714256}, {'id': 4, 'sentence': 'relatează BBC.', 'score': 1.6566044331920002}, {'id': 5, 'sentence': 'Este un moment de cotitură pentru Ucraina', 'score': 2.2102535776209726}, {'id': 6, 'sentence': 'a cărei situație pe front era tot mai dificilă și mai vulnerabilă.', 'score': 1.850462901492947}, {'id': 7, 'sentence': 'Rezultatul final al votului a fost de 311 la 112', 'score': 1.4909439898728003}, {'id': 8, 'sentence': 'voturile contra fiind ale republicanilor trumpiști.', 'score': 2.0707555414900005}, {'id': 9, 'sentence': 'Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului.', '

In [80]:
def summary(sentences_data):
    cnt = 0
    summary = ""
    for sentence_score in sentences_data:
        cnt += sentence_score["score"]
    avg = cnt / len(sentences_data)
    for sentence_score in sentences_data:
        if sentence_score["score"] >= avg:
                summary += sentence_score["sentence"] + " "
    return summary

# Example usage
summary = summary(sentence_scores)
print("Sumar: ", summary)
print("Text original: ", article_text)
    

Sumar:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia a fost aprobat după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front Este un moment de cotitură pentru Ucraina Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului. Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare urmând să fie Conform oficialilor americani familiarizați cu subiectul Preşedintele Joe Biden a salutat Camera Reprezentanţilor a adoptat și un nou pachet de sprijin militar de mai multe miliarde de dolari pentru Israel care este în război cu teroriștii Hamas în ciuda preocupărilor internaţionale cu privire la situaţia civililor din Gaza. Cele două ajutoare trebuie aprobate de Senat 
Text original:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, sâmbătă seară,

In [81]:
# Print sentences with their scores descending after score
def print_sentences_with_scores(sentences_data):
    sorted_sentences = sorted(sentences_data, key=lambda x: x["score"], reverse=True)
    for sentence in sorted_sentences:
        print(f"Score: {sentence['score']} - Sentence: {sentence['sentence']}")
        
# Example usage
print_sentences_with_scores(sentence_scores)

def print_sentences_in_order(sentences_data):
    sorted_sentences = sorted(sentences_data, key=lambda x: x["id"])
    for sentence in sorted_sentences:
        print(f"Sentence: {sentence['sentence']}")
        
# Example usage
print_sentences_in_order(sentence_scores)

Score: 2.4849066497880004 - Sentence: a fost aprobat
Score: 2.4849066497880004 - Sentence: urmând să fie
Score: 2.4849066497880004 - Sentence: Conform oficialilor americani familiarizați cu subiectul
Score: 2.4849066497880004 - Sentence: care este în război cu teroriștii Hamas
Score: 2.4849066497880004 - Sentence: aprobate de Senat
Score: 2.3647470279714256 - Sentence: după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front
Score: 2.3497516137519456 - Sentence: Cele două ajutoare trebuie
Score: 2.282174095733918 - Sentence: Preşedintele Joe Biden a salutat
Score: 2.2716380441681356 - Sentence: Camera Reprezentanţilor a adoptat și un nou pachet de sprijin militar de mai multe miliarde de dolari pentru Israel
Score: 2.2428698369229574 - Sentence: Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare
Score: 2.2102535776209726 - Sentence: Este un moment de cotitură pentru Ucraina
Score: 2.174

In [82]:
def add_punctuation(summary):
    doc = nlp(summary)
    complete_sentences = []
    buffer = []
    
    for sent in doc.sents:
        sentence = sent.text.strip()
        if sentence:
            buffer.append(sentence)
        if sentence.endswith('.'):
            complete_sentences.append(' '.join(buffer))
            buffer = []

    if buffer:
        complete_sentences.append(' '.join(buffer) + '.')

    return ' '.join(complete_sentences)

# Example usage
summary_with_punctuation = add_punctuation(summary)
print("Sumar cu punctuație: ", summary_with_punctuation)
print("Text original: ", article_text)

Sumar cu punctuație:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia a fost aprobat după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front Este un moment de cotitură pentru Ucraina Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului. Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare urmând să fie Conform oficialilor americani familiarizați cu subiectul Preşedintele Joe Biden a salutat Camera Reprezentanţilor a adoptat și un nou pachet de sprijin militar de mai multe miliarde de dolari pentru Israel care este în război cu teroriștii Hamas în ciuda preocupărilor internaţionale cu privire la situaţia civililor din Gaza. Cele două ajutoare trebuie aprobate de Senat.
Text original:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, 

THIRD APPROACH WITH MORE PREPROCESSING, LEMMATIZING AND ANALYSIS

In [83]:
def preprocess_text(text):
    patterns = [
        r'\bData publicării:.*?\d{2}:\d{2}\b',
        r'\bFoto:[^\n]+\b',
        r'Editor\s*:[^\n]+\b',
        r'Etichete:[^\n]+',
        r'Urmărește știrile[^\n]+',
        r'\b(Top Citite|Top Articole)[^\n]+',
        r'Sursa:[^\n]+',
        r'([Aa]scultă|Citește) și:[^\n]+',
        r'\bVezi și:[^\n]+',
        r'[\r\n]+\s*[\r\n]+'
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.DOTALL)

    # Lemmatizare și corectare ortografică
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_punct and not token.is_stop])

    return lemmatized_text

In [84]:
def tokenize_and_analyze(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents if len(sent.text.strip().split()) > 5]
    return sentences

In [85]:
from collections import Counter

def compute_tfidf(sentences):
    word_counts = Counter(word.lower() for sentence in sentences for word in sentence.split())
    tf = {word: count/len(word_counts) for word, count in word_counts.items()}
    idf = {word: math.log(len(sentences) / (1 + sum(1 for s in sentences if word in s))) for word in word_counts}
    tfidf = {word: tf[word]*idf[word] for word in word_counts}
    return tfidf

In [86]:
def select_sentences_for_summary(sentences, tfidf_scores, threshold=0.1):
    summary_sentences = []
    for sentence in sentences:
        sentence_score = sum(tfidf_scores.get(word.lower(), 0) for word in sentence.split())
        if sentence_score > threshold:
            summary_sentences.append(sentence)
    return summary_sentences

In [87]:
def generate_summary(url):
    article_text = fetch_article(url)
    preprocessed_text = preprocess_text(article_text)
    sentences = tokenize_and_analyze(preprocessed_text)
    tfidf_scores = compute_tfidf(sentences)
    summary = select_sentences_for_summary(sentences, tfidf_scores)
    return " ".join(summary)

In [88]:
url = "https://www.digi24.ro/stiri/externe/congresul-a-aprobat-ajutorul-militar-critic-de-61-de-miliarde-pentru-ucraina-2767383"
summary = generate_summary(url)
print("Sumar: ", summary)
print("Text original: ", article_text)


Sumar:  
Text original:  Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, sâmbătă seară, de Camera Reprezentanților din Congresul SUA, după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front, relatează BBC. Este un moment de cotitură pentru Ucraina, a cărei situație pe front era tot mai dificilă și mai vulnerabilă. Rezultatul final al votului a fost de 311 la 112, voturile contra fiind ale republicanilor trumpiști. Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului. Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare, urmând să fie, apoi, promulgat de președintele Joe Biden. Conform oficialilor americani familiarizați cu subiectul, o parte din ajutoare vor ajunge pe frontul din Ucraina în cel mult o săptămână. Preşedintele Joe Biden a salutat, sâmbătă, adopta

Graph based summarization method

In [ ]:
# Create a folder for the news articles and summaries

import os
from datasets import load_dataset

ro_dataset = load_dataset("readerbench/ro-text-summarization")
ro_dataset["train"][5]
# get total number of ro_dataset

number_of_articles = len(ro_dataset["train"])
for line in ro_dataset["train"]:
    category = line["Category"]
    content = line["Content"]
    summary = line["Summary"]
    if category not in os.listdir("romanian_news/news_articles"):
        os.makedirs(f"romanian_news/news_articles/{category}")
    if category not in os.listdir("romanian_news/summaries"):
        os.makedirs(f"romanian_news/summaries/{category}")
    # create a file for content in news_articles and summary in summaries, name it 000...n.txt, and get the number of digits we need from number_of_articles
    txt_name_format = f"{{:0{len(str(number_of_articles))}d}}.txt"
    i = len(os.listdir(f"romanian_news/news_articles/{category}")) + 1
    with open(f"romanian_news/news_articles/{category}/{txt_name_format.format(i)}", "w", encoding="utf-8") as file:
        file.write(content)
    j = len(os.listdir(f"romanian_news/summaries/{category}")) + 1
    with open(f"romanian_news/summaries/{category}/{txt_name_format.format(j)}", "w", encoding="utf-8") as file:
        file.write(summary)

In [89]:
import pandas as pd

paths = []
filenames = []
categories = []
article_or_summary_type = []

for path, _, filenames_in_path in os.walk('input_data'):
    for filename in filenames_in_path:
        paths.append(os.path.join(path, filename))
        filenames.append(filename)
        categories.append(path.split("\\")[-1])
        article_or_summary_type.append(path.split("\\")[-2])
        
print(paths)
print(filenames)
print(categories)
print(article_or_summary_type)

['input_data\\BBC News Summary\\News Articles\\business\\001.txt', 'input_data\\BBC News Summary\\News Articles\\business\\002.txt', 'input_data\\BBC News Summary\\News Articles\\business\\003.txt', 'input_data\\BBC News Summary\\News Articles\\business\\004.txt', 'input_data\\BBC News Summary\\News Articles\\business\\005.txt', 'input_data\\BBC News Summary\\News Articles\\business\\006.txt', 'input_data\\BBC News Summary\\News Articles\\business\\007.txt', 'input_data\\BBC News Summary\\News Articles\\business\\008.txt', 'input_data\\BBC News Summary\\News Articles\\business\\009.txt', 'input_data\\BBC News Summary\\News Articles\\business\\010.txt', 'input_data\\BBC News Summary\\News Articles\\business\\011.txt', 'input_data\\BBC News Summary\\News Articles\\business\\012.txt', 'input_data\\BBC News Summary\\News Articles\\business\\013.txt', 'input_data\\BBC News Summary\\News Articles\\business\\014.txt', 'input_data\\BBC News Summary\\News Articles\\business\\015.txt', 'input_da

In [90]:
df = pd.DataFrame({
    'path': paths,
    'filename': filenames,
    'category': categories,
    'article_or_summary': article_or_summary_type
})

df.to_csv('input_data.csv', index=False)
df

,path,filename,category,article_or_summary
0,input_data\BBC News Summary\News Articles\busi...,001.txt,business,News Articles
1,input_data\BBC News Summary\News Articles\busi...,002.txt,business,News Articles
2,input_data\BBC News Summary\News Articles\busi...,003.txt,business,News Articles
3,input_data\BBC News Summary\News Articles\busi...,004.txt,business,News Articles
4,input_data\BBC News Summary\News Articles\busi...,005.txt,business,News Articles
...,...,...,...,...
4445,input_data\BBC News Summary\Summaries\tech\397...,397.txt,tech,Summaries
4446,input_data\BBC News Summary\Summaries\tech\398...,398.txt,tech,Summaries
4447,input_data\BBC News Summary\Summaries\tech\399...,399.txt,tech,Summaries
4448,input_data\BBC News Summary\Summaries\tech\400...,400.txt,tech,Summaries


In [91]:
import plotly_express as pe
import cufflinks as cf

cf.go_offline() # required to use plotly offline (no account required).

In [92]:
from collections import Counter

def count_number_of_articles_per_category(df):
    return Counter(df[df['article_or_summary'] == 'News Articles']['category'])

counter = count_number_of_articles_per_category(df)
counter

pd.DataFrame(counter.items(), columns=['Category', 'Number of articles']).iplot(kind='bar', x='Category', y='Number of articles', title='Number of articles per category')

In [93]:
from nltk.tokenize import sent_tokenize

def read_article(text):
    sentences = []
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence.replace("[^a-zA-Z0-9]", " ")
        # print(sentence)
        # print()
    return sentences
        
read_article(article_text)

['Pachetul de ajutor militar pentru Ucraina în războiul său de apărare cu Rusia, de aproximativ 61 de miliarde de dolari, a fost aprobat, sâmbătă seară, de Camera Reprezentanților din Congresul SUA, după luni întregi de tergiversări și în contextul unei situații extrem de complicate pentru apărătorii ucraineni pe front, relatează BBC.',
 'Este un moment de cotitură pentru Ucraina, a cărei situație pe front era tot mai dificilă și mai vulnerabilă.',
 'Rezultatul final al votului a fost de 311 la 112, voturile contra fiind ale republicanilor trumpiști.',
 'Democrații au votat alături de o parte a conferinței republicane pentru deblocarea ajutorului.',
 'Este de așteptat ca Senatul SUA să aprobe și el pachetul de ajutor în zilele următoare, urmând să fie, apoi, promulgat de președintele Joe Biden.',
 'Conform oficialilor americani familiarizați cu subiectul, o parte din ajutoare vor ajunge pe frontul din Ucraina în cel mult o săptămână.',
 'Preşedintele Joe Biden a salutat, sâmbătă, adopt

In [94]:
file_path_example = df.iloc[0]['path']
with open(file_path_example, 'r', encoding='utf-8') as file:
    text = file.read()
    
text_sentences_tokenized = read_article(text)
text_sentences_tokenized

['Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.',
 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.',
 'TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.',
 'Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.',
 'Time Warner said on Friday that it now owns 8% of search-engine Google.',
 'But its own internet business, AOL, had has mixed fortunes.',
 'It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.',
 "However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.",
 "It hopes to increase subscribers by offering the online service free to TimeWarner internet cus

In [95]:
from textblob import TextBlob

mod_sent = []
for tok in text_sentences_tokenized:
    blob = TextBlob(tok)
    correct_sent = str(blob.correct())
    print("Original: ", tok)
    print("Corrected: ", correct_sent)
    print()
    mod_sent.append(correct_sent)
    
" ".join(mod_sent)

Original:  Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.
Corrected:  D sales boost Time Earner profit

Quarterly profits at of media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

Original:  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales.
Corrected:  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher avert sales.

Original:  TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.
Corrected:  TimeWarner said fourth quarter sales rose 2% to $11.in from $10.in.

Original:  Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.
Corrected:  Its profits were buoyed by one-o

'D sales boost Time Earner profit\n\nQuarterly profits at of media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher avert sales. TimeWarner said fourth quarter sales rose 2% to $11.in from $10.in. Its profits were buoyed by one-off gains which offset a profit dip at Earner Gros, and less users for AOL. Time Earner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL\'

In [96]:
import tensorflow_hub as hub
import numpy as np

In [97]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

def sentence_similarity(sentence1, sentence2, embed):
    x = embed([sentence1])[0]
    y = embed([sentence2])[0]
    return 1 - (np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y)))

In [98]:
print("sentence 1: ", mod_sent[0])
print("sentence 2: ", mod_sent[1])

print("Similarity: ", sentence_similarity(mod_sent[0], mod_sent[1], embed))

sentence 1:  D sales boost Time Earner profit

Quarterly profits at of media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.
sentence 2:  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher avert sales.
Similarity:  0.6398463845252991


In [99]:
def similarity_matrix(sentences, embed):
    n = len(sentences)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            matrix[i][j] = sentence_similarity(sentences[i], sentences[j], embed)
    return matrix

sim_matrix = similarity_matrix(mod_sent, embed)
sim_matrix

array([[-1.19209290e-07,  6.39846385e-01,  6.30399376e-01,
         6.02631181e-01,  8.14223900e-01,  9.14022103e-01,
         6.20975792e-01,  6.63688242e-01,  7.07335055e-01,
         8.91179942e-01,  7.24949718e-01,  5.94046921e-01,
         6.26118273e-01,  9.47235677e-01,  6.89721346e-01,
         8.13311398e-01,  8.38467270e-01,  8.71464491e-01,
         6.82516158e-01,  7.21438169e-01],
       [ 6.39846385e-01,  0.00000000e+00,  7.57411435e-01,
         6.17585182e-01,  8.80824044e-01,  7.02536851e-01,
         9.23527196e-01,  6.81970298e-01,  5.46405494e-01,
         8.65049347e-01,  8.41560632e-01,  7.92188540e-01,
         7.90884718e-01,  7.57571757e-01,  6.75543904e-01,
         7.62117058e-01,  7.74449378e-01,  8.60018671e-01,
         6.80165499e-01,  6.48043871e-01],
       [ 6.30399376e-01,  7.57411435e-01, -1.19209290e-07,
         8.50433454e-01,  8.78451146e-01,  8.75006638e-01,
         6.03710920e-01,  7.81530529e-01,  8.07515278e-01,
         8.72011676e-01,  6.6

In [100]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, HoverTool
from bokeh.plotting import figure, from_networkx

import networkx as nx

In [101]:
output_notebook()

g = nx.Graph()

for i in range(sim_matrix.shape[0]):
    for j in range(sim_matrix.shape[1]):
        if i != j and sim_matrix[i][j] > 0.9:
            g.add_edge(i, j, weight=sim_matrix[i][j])
            
HOVER_TOOLTIPS = [("sent_tok", "@index")]
plot = figure(title="Sentence Similarity Graph", x_range = Range1d(-10.1, 10.1), y_range = Range1d(-10.1, 10.1),
                tools="pan,wheel_zoom,save,reset,hover", tooltips=HOVER_TOOLTIPS, active_scroll="wheel_zoom")

network_graph = from_networkx(g, nx.spring_layout, scale=7, center=(0, 0))
network_graph.node_renderer.glyph = Circle(radius=0.1, fill_color='skyblue')
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
plot.renderers.append(network_graph)

show(plot)

Loading BokehJS ...

In [102]:
file_path_summary = df[df['article_or_summary'] == 'Summaries'].iloc[0]['path']
with open(file_path_summary, 'r', encoding='utf-8') as file:
    summary_text = file.read()

In [103]:
def generate_summary(text, top_n_sentences, embeds):
    summarize_text = []
    sentences = read_article(text)
    sentence_similarity_martix = similarity_matrix(sentences, embeds)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    for i in range(top_n_sentences):
        summarize_text.append("".join(ranked_sentences[i][1]))
    return " ".join(summarize_text)

summary_generated = generate_summary(text, 5, embed)
print("Generated summary: ", summary_generated)
print("Original summary: ", summary_text)

Generated summary:  "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.
Original summary:  TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly p

In [104]:
import nltk

hypothesis = summary_generated
reference = summary_text
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print("BLEU Score: ", BLEUscore)

BLEU Score:  0.4453393957078566


In [105]:
print(sentence_similarity(summary_text, summary_generated, embed))

0.4166654348373413


In [106]:
from datasets import load_dataset

ro_dataset = load_dataset("readerbench/ro-text-summarization")

In [107]:
ro_dataset["train"][5]

{'Category': 'guvern',
 'Title': 'Moţiunea simplă „Tudorel Toader şi PSD-ALDE, luaţi mâinile murdare de pe Justiţie!”, dezbătută astăzi în Camera Deputaţilor',
 'Content': 'Liderul PSD, Liviu Dragnea, a fost întrebat de jurnalişti, luni, dacă ministrul Justiţiei trebuie să aibă emoţii la moţiunea simplă, iar preşedintele Camerei a răspuns: "Sincer, nu pot să vă răspund la această întrebare, pentru că nu ştiu”."Nu ştiu, chiar nu ştiu să vă răspund. Sincer, nu pot să vă răspund la această întrebare, pentru că nu ştiu”, a spus Dragnea zâmbind. Întrebat în continuare dacă Tudorel Toader are susţinerea grupului PSD, Liviu Dragnea a răspuns: "Vedem miercuri (când se votează moţiunea simplă - n. r.)”.Cu o zi în urmă, ministrul Justiţiei şi-a publicat bilanţul la doi ani de mandat, un document de 85 de pagini, din care aproape 20 sunt anexe preponderent referitoare la lucrările de investiţii din peninteniciare. Ministrul enumeră conferinţele cu privire la sistemul judiciar organizate, prezintă

In [111]:
# get total number of ro_dataset

number_of_articles = len(ro_dataset["train"])

In [116]:
for line in ro_dataset["train"]:
    category = line["Category"]
    content = line["Content"]
    summary = line["Summary"]
    if category not in os.listdir("romanian_news/news_articles"):
        os.makedirs(f"romanian_news/news_articles/{category}")
    if category not in os.listdir("romanian_news/summaries"):
        os.makedirs(f"romanian_news/summaries/{category}")
    # create a file for content in news_articles and summary in summaries, name it 000...n.txt, and get the number of digits we need from number_of_articles
    txt_name_format = f"{{:0{len(str(number_of_articles))}d}}.txt"
    i = len(os.listdir(f"romanian_news/news_articles/{category}")) + 1
    with open(f"romanian_news/news_articles/{category}/{txt_name_format.format(i)}", "w", encoding="utf-8") as file:
        file.write(content)
    j = len(os.listdir(f"romanian_news/summaries/{category}")) + 1
    with open(f"romanian_news/summaries/{category}/{txt_name_format.format(j)}", "w", encoding="utf-8") as file:
        file.write(summary)